# Evaluating gemma-7b-it for text classification
## Task: Identify if a tweet is about a "disaster"

### Setup - Using kaggle GPU P100

In [1]:
!pip install -q -U torch --index-url https://download.pytorch.org/whl/cu117

In [2]:
!pip install -q -U transformers=="4.38.2"
!pip install -q accelerate
!pip install -q -i https://pypi.org/simple/ bitsandbytes
!pip install -q -U datasets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.1 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.2 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which

In [3]:
!pip install -q -U git+https://github.com/huggingface/trl
!pip install -q -U git+https://github.com/huggingface/peft

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

import os
from tqdm import tqdm

import torch
import torch.nn as nn

import transformers
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from datasets import Dataset
from peft import LoraConfig, PeftConfig
import bitsandbytes as bnb
from trl import SFTTrainer

from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

2024-03-12 09:42:32.231855: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-12 09:42:32.231962: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-12 09:42:32.377981: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
# Read data
submission_path = '/kaggle/input/tweets/disaster_submission.csv'
train_path = '/kaggle/input/tweets/disaster_train.csv'
test_path = '/kaggle/input/tweets/disaster_test.csv'

submission_df = pd.read_csv(submission_path)
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [8]:
# Display column names
train_df.columns.tolist()

['id',
 'keyword',
 'location',
 'text',
 'target',
 'text_clean',
 'location_clean',
 'text_location',
 'text_location_clean',
 'class',
 'prompt_text',
 'prompt_text_test',
 'prompt_text_location',
 'prompt_text_location_test',
 'prompt_text_clean',
 'prompt_text_clean_test',
 'prompt_text_location_clean',
 'prompt_text_location_clean_test']

* If ***_text*** in name of prompt column - text is included.
* If ***_test*** in name of prompt column - there is no answer included.
* If ***_location*** in name of prompt column - location is included.
* If ***_clean*** in name of prompt column - prompt has been cleaned (see EDA).

In [9]:
# Split train_df into train, eval, test
# Keep test_df as held out for submission

# Sample equally from yes and no for train test
X_train = list()
X_test = list()
for target in ['yes', 'no']:
    train, test  = train_test_split(train_df[train_df['class']==target], 
                                    train_size=500,
                                    test_size=500, 
                                    random_state=1)
    X_train.append(train)
    X_test.append(test)

X_train = pd.concat(X_train).sample(frac=1, random_state=10) # Shuffle
X_test = pd.concat(X_test)

# Balanced eval set
eval_mask = ~train_df.index.isin(X_train.index) & ~train_df.index.isin(X_test.index)
X_eval = train_df[eval_mask].groupby('class').apply(
    lambda x: x.sample(n=100, random_state=1, replace=True)).reset_index(drop=True)

# Reset index for training data
X_train = X_train.reset_index(drop=True)

y_true = X_test['class']

train_data = Dataset.from_pandas(X_train)
eval_data = Dataset.from_pandas(X_eval)

In [10]:
# Check sizes
print(X_train.shape, X_eval.shape, X_test.shape, test_df.shape)

(1000, 18) (200, 18) (1000, 18) (3263, 11)


Train
* prompt_text - prompt with just text
* prompt_text_location - prompt with text and location
* prompt_text_clean - prompt with cleaned text
* prompt_text_location_clean - prompt with cleaned text and location


Test - Same but without model answers
* prompt_text_test
* prompt_text_location_test
* prompt_text_clean_test
* prompt_text_location_clean_test

In [11]:
# Load model + quantization
model_name = "/kaggle/input/gemma/transformers/7b-it/1"

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config, 
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [12]:
# Test of 1
print(X_test.iloc[2][['text', 'class']])
prompt = X_test.iloc[2]['prompt_text_test']
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**input_ids, max_new_tokens=1, temperature=0.0)
result = tokenizer.decode(outputs[0])
answer = result.split("model\n")[-1].lower()
print('\nPrediction: '+answer)

text     @Chibi877 --head. It hit the wall behind him with a loud bang. 'Language!' Drake shouted at him before getting up. 'I'm going out stay--
class                                                                                                                                         yes
Name: 4658, dtype: object

Prediction: yes


In [13]:
def predict(X_test, prompt_col, model, tokenizer):
    y_pred = []
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i][prompt_col]
        input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(**input_ids, max_new_tokens=1, temperature=0.0)
        result = tokenizer.decode(outputs[0])
        answer = result.split("model\n")[-1].lower()
        if answer == 'no':
            y_pred.append('no')
        elif answer == 'yes':
            y_pred.append('yes')
        else:
            y_pred.append('none')
    return y_pred

In [14]:
def evaluate(y_true, y_pred):
    labels = ['yes', 'no']
    mapping = {'yes': 1, 'no': 0, 'none':-1}
    def map_func(x):
        return mapping.get(x, 1)
    
    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')
    
    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels
    
    for label in unique_labels:
        label_indices = [i for i in range(len(y_true)) 
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')
        
    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)
    
    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2])
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [15]:
# Zero shot learning on prompt_text
y_pred = predict(X_test, 'prompt_text_test', model, tokenizer)
evaluate(y_true, y_pred)

100%|██████████| 1000/1000 [06:46<00:00,  2.46it/s]

Accuracy: 0.702
Accuracy for label 0: 0.716
Accuracy for label 1: 0.688

Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.72      0.71       500
           1       0.71      0.69      0.70       500

    accuracy                           0.70      1000
   macro avg       0.70      0.70      0.70      1000
weighted avg       0.70      0.70      0.70      1000


Confusion Matrix:
[[358 142   0]
 [156 344   0]
 [  0   0   0]]


In [16]:
# Zero shot learning on prompt_text_location
y_pred = predict(X_test, 'prompt_text_location_test', model, tokenizer)
evaluate(y_true, y_pred)

100%|██████████| 1000/1000 [07:02<00:00,  2.37it/s]

Accuracy: 0.693
Accuracy for label 0: 0.806
Accuracy for label 1: 0.580

Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.81      0.72       500
           1       0.75      0.58      0.65       500

    accuracy                           0.69      1000
   macro avg       0.70      0.69      0.69      1000
weighted avg       0.70      0.69      0.69      1000


Confusion Matrix:
[[403  97   0]
 [210 290   0]
 [  0   0   0]]


In [17]:
# Zero shot learning on prompt_text_clean
y_pred = predict(X_test, 'prompt_text_clean_test', model, tokenizer)
evaluate(y_true, y_pred)

100%|██████████| 1000/1000 [06:26<00:00,  2.59it/s]

Accuracy: 0.702
Accuracy for label 0: 0.732
Accuracy for label 1: 0.672

Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.73      0.71       500
           1       0.71      0.67      0.69       500

    accuracy                           0.70      1000
   macro avg       0.70      0.70      0.70      1000
weighted avg       0.70      0.70      0.70      1000


Confusion Matrix:
[[366 134   0]
 [164 336   0]
 [  0   0   0]]


In [18]:
# Zero shot learning on prompt_text_location_clean
y_pred = predict(X_test, 'prompt_text_location_clean_test', model, tokenizer)
evaluate(y_true, y_pred)

100%|██████████| 1000/1000 [06:46<00:00,  2.46it/s]

Accuracy: 0.695
Accuracy for label 0: 0.824
Accuracy for label 1: 0.566

Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.82      0.73       500
           1       0.76      0.57      0.65       500

    accuracy                           0.69      1000
   macro avg       0.71      0.69      0.69      1000
weighted avg       0.71      0.69      0.69      1000


Confusion Matrix:
[[412  88   0]
 [217 283   0]
 [  0   0   0]]


* The best prompt for zero-shot learning is not necessarily the best prompt for fine-tuning;
* However, for simplicity, we will continue with prompt_text.

# Fine tuning

In [19]:
# Fine tuning
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules="all-linear",
)

training_arguments = TrainingArguments(
    output_dir="logs",
    num_train_epochs= 4,
    gradient_checkpointing=True,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=1e-5, # 2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=False,
    evaluation_strategy='steps',
    eval_steps = 112,
    eval_accumulation_steps=1,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    load_best_model_at_end=True
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="prompt_text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
    max_seq_length=1024,
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [20]:
# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained("trained-model")

Step,Training Loss,Validation Loss
112,5.949400,4.773743
224,1.954300,2.579322
336,1.723400,2.509944
448,1.669900,2.465275


In [21]:
# predict and evaluate
y_pred = predict(X_test, 'prompt_text_test', model, tokenizer)
evaluate(y_true, y_pred)

100%|██████████| 1000/1000 [07:27<00:00,  2.23it/s]

Accuracy: 0.778
Accuracy for label 0: 0.800
Accuracy for label 1: 0.756

Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.80      0.78       500
           1       0.79      0.76      0.77       500

    accuracy                           0.78      1000
   macro avg       0.78      0.78      0.78      1000
weighted avg       0.78      0.78      0.78      1000


Confusion Matrix:
[[400 100   0]
 [122 378   0]
 [  0   0   0]]


In [22]:
test_df.iloc[[40]]

,id,keyword,location,text,text_clean,location_clean,text_location,text_location_clean,prompt_text_test,prompt_text_location_test,prompt_text_clean_test
40,125,accident,"Frankfurt, Germany",@DaveOshry @Soembie So if I say that I met her by accident this week- would you be super jelly Dave? :p,@DaveOshry @Soembie So if I say that I met her by accident this week would you be super jelly Dave? p,"Frankfurt, Germany","Location: Frankfurt, Germany. Tweet: @DaveOshry @Soembie So if I say that I met her by accident this week- would you be super jelly Dave? :p","Location: Frankfurt, Germany. Tweet: @DaveOshry @Soembie So if I say that I met her by accident this week would you be super jelly Dave? p",NaN,NaN,NaN


In [23]:
# Save submission
y_pred = predict(test_df, 'prompt_text_test', model, tokenizer)
submission_df['target'] = y_pred
# If fail, default to yes (to maximise sub score)
submission_df['target'] = submission_df['target'].apply(lambda x: 0 if x == 'no' else 1)
submission_df.to_csv('submission1.csv', index=False)

  1%|          | 40/3263 [00:15<21:27,  2.50it/s]


ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
# Save model
torch.save(model.state_dict(), '/kaggle/working/gemma_7b_it_disaster_tweets.pth')